In [1]:

%pprint

Pretty printing has been turned OFF



---
# Load needed libraries and functions

In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%matplotlib inline
from datetime import datetime
from neo4j.exceptions import ServiceUnavailable
import humanize
import matplotlib.pyplot as plt
import time
import winsound

bin_count = 12
duration = 1000  # milliseconds
freq = 880  # Hz
height_inches = 3.0
width_inches = 18.0

In [4]:

t0 = time.time()

# Get the Neo4j driver
from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

# Get the neo4j object
from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

try:
    
    version_str = cu.driver.get_server_info().agent
    print(f'======== {version_str} ========')
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)
    
    # 400 6,094 37 minutes and 50 seconds
    # 800 6,094 36 minutes and 42 seconds
    # 1,600 8,349 59 minutes and 6 seconds
    # 3_200 9,974 1 hour, 4 minutes and 24 seconds
    # 6_400 10,635 1 hour, 23 minutes and 11 seconds
    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    lru.build_isheader_logistic_regression_elements(verbose=False)
    lru.build_isqualified_logistic_regression_elements(sampling_strategy_limit=5_000, verbose=False)
    lru.build_pos_logistic_regression_elements(sampling_strategy_limit=6_400, verbose=True)
    
    from crf_utils import CrfUtilities
    crf = CrfUtilities(ha=ha, hc=hc, cu=cu, lru=lru, verbose=True)
    
    from section_utils import SectionUtilities
    su = SectionUtilities(s=s, ha=ha, cu=cu, crf=crf, verbose=False)
    
    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    print('You need to start Neo4j as a console')
    raise
except Exception as e:
    print(f'{e.__class__}: {str(e).strip()}')
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
winsound.Beep(freq, duration)
print(f'Utility libraries created in {duration_str}')
print(f'Last run on {datetime.now()}')

======== Neo4j/4.4.7 ========
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
I have 10,635 labeled parts of speech in here
Got this <class 'numpy.core._exceptions._ArrayMemoryError'> error in build_pos_logistic_regression_elements trying  to turn the pos_symbol TF-IDF matrix into a normal array: Unable to allocate 31.7 GiB for an array with shape (10635, 400406) and data type float64
Utility libraries created in 1 hour, 5 minutes and 3 seconds
Last run on 2023-02-25 22:54:31.530484



---
# Training

In [21]:

# I have 9,340 hand-labeled qualification strings in here
# Is-qualified classifer retrained in 1 minute and 14 seconds
t0 = time.time()
basic_quals_dict = lru.sync_basic_quals_dict(sampling_strategy_limit=10_000, verbose=False)
lru.retrain_isqualified_classifier(verbose=True)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
print(f'Is-qualified classifer retrained in {duration_str}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
I have 10,856 hand-labeled qualification strings in here
Retraining complete
Is-qualified classifer retrained in 2 minutes and 43 seconds


In [22]:

from pandas import DataFrame

cypher_str = f'''
    MATCH (fn:FileNames)
    WHERE
        fn.percent_fit < 0.4 AND
        ((fn.is_closed IS NULL) OR (fn.is_closed = false)) AND
        ((fn.is_verified IS NULL) OR (fn.is_verified = false)) AND
        ((fn.is_opportunity_application_emailed IS NULL) OR
        (fn.is_opportunity_application_emailed = false))
    RETURN
        fn.percent_fit AS percent_fit,
        fn.file_name AS file_name,
        fn.posting_url AS url
    ORDER BY fn.percent_fit ASC;'''
row_objs_list = []
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
files_list = []
if row_objs_list:
    files_list = DataFrame(row_objs_list).file_name.tolist()
print(f'Only {len(files_list)} more mis-estimated minimum-requirements-met percentages to go!')

Only 3133 more mis-estimated minimum-requirements-met percentages to go!



----

In [23]:

import os

file_name = '1be7cd43854ea836_Python_Developer_with_AWS_Atlanta_GA_Indeed_com.html'
# file_name = files_list.pop()
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
page_soup = wsu.get_page_soup(file_path)
div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
child_strs_list = ha.get_navigable_children(div_soup, [])
cu.ensure_filename(file_name, verbose=False)
cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
print(file_name)

a7f9b8bd279918d6_Junior_Data_Analyst_Engineer_Remote_Indeed_com.html


In [24]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
is_header_list = []
for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
    if is_header is None:
        probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
        idx = probs_list.index(max(probs_list))
        is_header = [True, False][idx]
    is_header_list.append(is_header)
feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_tuple_list = []
for feature_dict in feature_dict_list:
    feature_tuple_list.append(hc.get_feature_tuple(feature_dict, lru.pos_lr_predict_single))
crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)
winsound.Beep(freq, duration)

['H-CS', 'H-CS', 'O-CS', 'O-CS', 'O-CS', 'O-TS', 'O-SP', 'H-TS', 'H-TS', 'H-TS', 'O-TS', 'O-IP', 'O-TS', 'O-TS', 'O-IP', 'H-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'H-TS', 'H-CS', 'O-CS', 'H-CS', 'O-CS', 'H-CS', 'O-CS', 'H-CS', 'O-CS', 'H-CS', 'O-CS', 'O-RQ', 'O-IP', 'H-SP', 'O-O', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP']
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 39]


[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 39]


In [ ]:
raise

In [35]:

# Display the context of an individual child string
idx = 15
print(indices_list); child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]; basic_quals_dict = s.load_object('basic_quals_dict')
print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end=''); print(f'{idx} {pos_symbol}) {child_str}')

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 39]
15 H-RQ) WHO YOU ARE


In [13]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<b>Novato, CA (2-3 days a week Hybrid)</b>" in basic_quals_dict: 0


In [34]:

def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = """MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        SET
            np.is_header = 'False',
            np.is_task_scope = 'True',
            np.is_minimum_qualification = 'False',
            np.is_preferred_qualification = 'False',
            np.is_educational_requirement = 'False',
            np.is_legal_notification = 'False',
            np.is_other = 'False',
            np.is_corporate_scope = 'False',
            np.is_job_title = 'False',
            np.is_office_location = 'False',
            np.is_job_duration = 'False',
            np.is_supplemental_pay = 'False',
            np.is_interview_procedure = 'False',
            np.is_posting_date = 'False'
        """ + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
row_objs_list

[{'navigable_parent': '<li>You will use problem solving and creativity to apply appropriate metrics in the creation of compelling and thorough analysis.</li>', 'is_header': 'False', 'is_task_scope': 'True', 'is_qualification': None, 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'False'}]

In [36]:

# Show what's in the database already for this html string
def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = '''MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        ''' + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
row_objs_list

[{'navigable_parent': 'WHO YOU ARE', 'is_header': 'True', 'is_task_scope': 'False', 'is_qualification': None, 'is_minimum_qualification': 'True', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'False'}]

In [ ]:

# Remove this particular child string from the quals dictionary and database
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = child_strs_list[idx]
basic_quals_dict.pop(child_str, None)
# basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')
def do_cypher_tx(tx, qualification_str, verbose=False):
    cypher_str = '''
        MATCH (qs:QualificationStrings {qualification_str: $qualification_str})
        DETACH DELETE qs;
        '''
    results_list = tx.run(query=cypher_str, parameters={'qualification_str': qualification_str})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, qualification_str=child_str, verbose=False)


----

In [37]:

# You've made no changes to the qualification dictionary (regardless of parts-of-speech changes)
from IPython.display import clear_output

def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.is_verified = true
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    parameter_dict = {'file_name': file_name}
    results_list = tx.run(query=cypher_str, parameters=parameter_dict)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))

    return values_list
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=True)
row_objs_list


        MATCH (fn:FileNames {file_name: "a7f9b8bd279918d6_Junior_Data_Analyst_Engineer_Remote_Indeed_com.html"})
        SET fn.is_verified = true
        RETURN fn;


[{'fn': <Node element_id='972917' labels=frozenset({'FileNames'}) properties={'file_name': 'a7f9b8bd279918d6_Junior_Data_Analyst_Engineer_Remote_Indeed_com.html', 'percent_fit': 0.9333333333333333, 'posting_url': 'https://www.indeed.com/rc/clk/dl?jk=a7f9b8bd279918d6&from=ja&qd=RnZhMybXSk4M3QtTVGXWocPDA-jVn_f73KUcK2QrGXxWzxuTTZnceBTcgT1wk7VUOC2eXp93ksvkWfcXnoYGxFf_vzSr4VOn_GaBBjQ30gg&rd=R6q1JXX_v8hKS-84jS16xdPu-0bJyOrVXPgWOW9J5h0&tk=1gq74mat1m65h800&alid=63b02dca1ef86228dd5d5128', 'is_verified': True}>}]

In [20]:

# Mark the file name as needing retraining everywhere
import numpy as np
from IPython.display import clear_output

# file_name = '14885afaa7bbd01e_Software_Developer_Engineer_in_Test_SDET_Eagle_ID_83616_Indeed_com.html'
mask_series = lru.hunting_df.percent_fit.isin([file_name])
lru.hunting_df.loc[mask_series, 'percent_fit'] = np.nan
s.store_objects(hunting_df=lru.hunting_df)
def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.percent_fit = NULL, fn.is_verified = false
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    results_list = tx.run(query=cypher_str, parameters={'file_name': file_name})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=False)
row_objs_list

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\hunting_df.pkl


[{'fn': <Node element_id='972425' labels=frozenset({'FileNames'}) properties={'file_name': '77496bf0516f016b_BI_Developer_on_W2_Remote_Indeed_com.html', 'posting_url': 'https://www.indeed.com/rc/clk/dl?jk=77496bf0516f016b&from=ja&qd=RnZhMybXSk4M3QtTVGXWocPDA-jVn_f73KUcK2QrGXztZO_RkD_EE3kw2XARkt-trWGNkDafvEGVhl1f5NcfKl_MKnaSAFGAsD6kfERFt3g&rd=wRw4VfvjpFhMaapCW_-ezV_MKnaSAFGAsD6kfERFt3g&tk=1gq4kobfim65h800&alid=63b02dca1ef86228dd5d5128', 'is_verified': False}>}]

In [ ]:

# Mark the file name as closed
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.is_closed = true
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

In [ ]:

# Manually label the unscored qual
qualification_str = quals_list[13]
print(qualification_str)
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[qualification_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)

In [ ]:

# Remove file name from database
# file_name = '3c031ea6ad293e92_General_Service_Technician_Westborough_MA_01581_Indeed_com.html'
cu.delete_filename_node(file_name, verbose=True)